## How the modelling process went:

Model | Train | Validation | Kaggle
:--- | :--- | :--- | :---
first logistic regression | 0.5929 | 0.5930 | 0.5950
first RF (100 trees) | 0.8560 | 0.7582 | 0.7743
first XGB (lr 0.2) | 0.8076 | 0.7768 | 0.7573
RF (300 trees) | 0.8567 | 0.7590 |
XGB (lr 0.05) | 0.8165 | 0.7802 | 0.7769 (top 67%)
-|-|-|-
smaller RF + imputation | 0.8557 | 0.7603 | 
smaller XGB + imputation | 0.8084 | 0.7790 | 
bigger RF + imputation | 0.8569 | 0.7611 | 
bigger XGB + imputation | 0.8137 | 0.7818 | 0.7743
-|-|-|-
smaller RF + interactions | 0.8656 | 0.7689 | 
smaller XGB + interactions | 0.8047 | 0.7764 | 
bigger RF + interactions | 0.8664 | 0.7695 | 0.7668
bigger XGB + interactions | 0.8132 | 0.7799 | 0.7770 (top 67%)
-|-|-|-
smaller RF + selection | 0.8578 | 0.7705 | 
smaller XGB + selection | 0.8045 | 0.7768 | 0.7742
bigger RF + selection | 0.8588 | 0.7710 | 0.7682
bigger XGB + selection | 0.8107 | 0.7803 | 0.7770
-|-|-|-
big RF tuned | 0.8502 | 0.7726 | 0.7702
big XGB tuned | 0.7931 | 0.7799 | 0.7778 (top 67%)
-|-|-|-
very big RF, tuned, all data | 0.8497 | - | 0.7708
very big XGB, tuned, all data | 0.7918 | - | 0.7783 (top 66%)

Tree-based ensemble models immediately outperformed logistic regression. In my experience XGBoost also almost always outperforms Random Forest and XGB can be improved more with tuning but I will try both for a while. A bigger XGB puts us in the top 67%. Validation AUC seems to be a good approximation of the public test set's AUC, but we need to take into account that a single validation set is not as reliable as crossvalidation. Crossvalidation is sadly currently inconvenient because to prevent overfitting and retain validation AUC as a decent estimate of leaderboard AUC, many features such as target encodings need to be created on train set only. 

Using sklearn.impute.IterativeImputer instead of random distibution sampling improved validation results a tiny bit but not the leaderboard score. Because it is almost certainly better than random values, I will count the kaggle score not improving as noise due the fact that it is calculated on only 25% of total test data. 

Adding >1000 feature products on top of that yielded very slightly better results on RF, but not on XGB (except for the most minor improvent on the leaderboard). Most features were not useful at all as shown by feature importances graphs so for both model goodness and time-efficiency we should try to pick out 150-200 features from the 1100+. Selecting a subset of features improved RF and XGB slightly so it was likely beneficial. Tuning and using all data obviously improved results, but as with previous changes, not nearly as much as I was expecting even though the optimal parameters were pretty different from the ones I used in the beginning. 

This was a very curious dataset, the task seemed very straightforward initially but i guess that was the reason why the leaderboard was incredibly densely packed and missing value imputation, feature interactions and tuning barely improved the end result. 


### Things still possible to try:

different sampling methods for uneven class distributions

multiple different models on differently engineered datasets, boosted together

different types and sizes of feature selection

In [1]:
develop = False

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score as AUC

from xgboost import XGBRegressor, plot_importance
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor

pd.set_option('display.max_columns', 100)

%matplotlib inline

figsize = (6,14)

def plot_importances(model, error=False, title=''):
    
    importances = model.feature_importances_
    indices = np.argsort(importances)#[::-1]

    fig, ax = plt.subplots(figsize=figsize)
    if error:
        std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
        plt.barh(range(train_X.shape[1]), importances[indices], xerr=std[indices], align="center")
    else:
        plt.barh(range(train_X.shape[1]), importances[indices], align="center")

    plt.yticks(range(train_X.shape[1]), [train_X.columns[ix] for ix in indices], rotation='horizontal')
    plt.ylim([-1, train_X.shape[1]])
    ax.xaxis.tick_top()
    plt.title(title, y=1.03)
    plt.show()

def plot_xgboost_importances(booster, model_name):
    fig, ax = plt.subplots(1,1,figsize=figsize)
    plot_importance(booster=booster, ax=ax, height=0.8, title=model_name+' - uses', importance_type='weight')
    plt.show()
    fig, ax = plt.subplots(1,1,figsize=figsize)
    plot_importance(booster=booster, ax=ax, height=0.8, title=model_name+' - avg gain', importance_type='gain')
    plt.show()
    fig, ax = plt.subplots(1,1,figsize=figsize)
    plot_importance(booster=booster, ax=ax, height=0.8, title=model_name+' - avg affected samples', 
                    importance_type='cover')
    plt.show()

def plot_tree_usefulness(model, validation_X, validation_y, model_name):
    predictions = []
    for tree in model.estimators_:
        predictions.append(tree.predict(validation_X))
        
    predictions = np.vstack(predictions)
    predictions = np.cumsum(predictions, axis=0)
    predictions = [predictions[i]/(i+1) for i in range(len(predictions))] #cum mean

    scores = []
    for pred in predictions:
        scores.append(AUC(validation_y, pred))
        
    plt.figure(figsize=(8, 4))
    plt.plot(scores, linewidth=3)
    plt.xlabel('trees')
    plt.ylabel('AUC')
    plt.title(model_name+' tree usefulnesses')
    plt.show()

In [2]:
# all features with big XGB model importance measure > 0.0012 (188/1136)
best_features = ['ord_2_x_ord_3', 'ord_0_x_ord_3', 'bin_0', 'ord_0_x_ord_2', 'ord_3_x_ord_5', 'ord_5_x_month', 
'bin_2_x_month', 'ord_0_x_month', 'ord_1_x_month', 'ord_0_x_ord_5', 'ord_2_x_month', 'ord_0_x_ord_4', 
'day_x_x_nom_1_Triangle', 'ord_2_x_ord_5', 'bin_2_x_ord_4', 'nom_1_target_mean', 'month_x_nom_4_Bassoon', 
'nom_8_target_mean', 'bin_0_target_mean', 'nom_3_target_mean', 'nom_9_target_mean', 'nom_7_target_mean', 
'ord_1_x_ord_3', 'ord_4_x_ord_5', 'ord_1_x_ord_4', 'day_target_mean', 'day_x_x_nom_3_India', 'bin_0_x_nom_3_lat', 
'bin_0_x_ord_5', 'ord_2', 'bin_1', 'ord_1_x_ord_2', 'month_x_nom_1_Trapezoid', 'bin_2_x_ord_3', 
'ord_4_x_nom_3_Russia', 'nom_5_target_mean', 'month_x_nom_3_Russia', 'ord_2_x_nom_4_wind', 'bin_2_x_ord_2', 
'nom_2_target_mean', 'month_target_mean', 'nom_1_Triangle', 'ord_4_x_nom_4_Bassoon', 'ord_0_x_ord_1', 
'bin_2_x_ord_5', 'bin_2_x_ord_0', 'ord_2_target_mean', 'ord_4_x_day_x', 'bin_2_x_ord_1', 'ord_3_x_nom_4_wind', 
'month_x_day_x', 'bin_2_x_nom_4_Bassoon', 'nom_4_Piano', 'ord_0_x_nom_4_Piano', 'ord_4_x_nom_1_Trapezoid', 
'bin_0_x_month_x', 'ord_3_x_nom_1_Trapezoid', 'nom_6_target_mean', 'ord_0_x_nom_4_Bassoon', 'bin_4_x_nom_4_Bassoon', 
'nom_4_target_mean', 'ord_3', 'ord_2_x_nom_1_Trapezoid', 'ord_3_x_nom_4_Bassoon', 'ord_3_x_nom_3_Russia', 
'bin_4_x_month', 'bin_2_x_bin_4', 'bin_0_x_month', 'ord_3_x_day_x', 'bin_1_x_day', 'day_x_x_nom_4_Bassoon', 
'ord_2_x_day_x', 'ord_2_x_nom_4_Bassoon', 'ord_4_x_nom_2_Lion', 'nom_0_Blue_x_nom_3_India', 'month_x_nom_2_Lion', 
'ord_0_x_nom_4_wind', 'nom_4_wind_x_nom_3_Russia', 'month_x_x_nom_0_Red', 'nom_8_was_na', 'ord_1_x_ord_5', 
'bin_1_x_month_x', 'bin_4_x_nom_0_Blue', 'ord_1_x_nom_2_Lion', 'nom_4_wind_x_nom_2_Lion', 'bin_0_x_ord_3', 
'ord_5_target_mean', 'ord_0_x_day_x', 'bin_4_x_ord_3', 'bin_1_x_nom_4_wind', 'bin_4_x_nom_3_Russia', 
'nom_0_Blue_x_nom_2_Lion', 'nom_1_Polygon_x_nom_4_Bassoon', 'bin_2_x_day_x', 'bin_4_x_day_x', 'ord_5_x_nom_3_Russia', 
'ord_0', 'ord_1_x_nom_4_Bassoon', 'bin_2_was_na', 'ord_4_x_nom_0_Blue', 'ord_2_x_nom_4_Piano', 'day_x_day_y', 
'ord_1_x_day_x', 'bin_2_x_nom_1_Trapezoid', 'ord_5_x_nom_4_Piano', 'bin_2_x_nom_0_Blue', 'ord_3_target_mean', 
'month_x_nom_0_Blue', 'bin_1_x_nom_4_Theremin', 'ord_0_x_nom_3_India', 'ord_1_x_nom_1_Trapezoid', 
'ord_5_x_nom_1_Trapezoid', 'ord_4_x_day_y', 'bin_4_x_ord_4', 'bin_0_x_ord_0', 'ord_4_target_mean', 
'ord_0_x_nom_2_Lion', 'day_x_x_nom_0_Blue', 'bin_2_x_nom_3_lat', 'ord_3_x_nom_4_Piano', 'ord_3_x_nom_0_Blue', 
'nom_9_was_na', 'ord_4_x_nom_4_wind', 'nom_3_Russia_x_nom_4_Bassoon', 'day_x_x_nom_1_Trapezoid', 
'day_y_x_nom_1_Triangle', 'day_y_x_nom_4_Bassoon', 'ord_3_was_na', 'ord_5_x_nom_4_Bassoon', 'bin_0_x_day', 
'bin_4_x_ord_1', 'day_x_x_nom_3_Russia', 'nom_1_Star_x_nom_4_Theremin', 'ord_2_x_nom_2_Lion', 'bin_4_x_nom_4_wind', 
'bin_0_x_ord_4', 'bin_2_x_nom_2_Lion', 'bin_1_x_ord_4', 'ord_2_x_nom_1_Star', 'month_y', 'ord_3_x_nom_2_Lion', 
'bin_4_x_ord_2', 'bin_1_x_ord_5', 'ord_2_x_ord_4', 'nom_1_Trapezoid_x_nom_4_Bassoon', 'nom_7_was_na', 
'month_x_x_nom_3_Finland', 'nom_0_Red_x_nom_4_Theremin', 'day_x_nom_1_Trapezoid', 'day_x_nom_4_Piano', 
'month_y_x_nom_0_Blue', 'nom_5_was_na', 'month_was_na', 'nom_3_Costa Rica_x_nom_4_Bassoon', 
'nom_3_lat_x_nom_1_Triangle', 'bin_3_x_nom_0_Red', 'bin_2_x_nom_3_Russia', 'day_x_nom_4_Bassoon', 
'ord_5_x_nom_3_Costa Rica', 'month_x_nom_4_Piano', 'day_x_x_nom_3_Costa Rica', 'ord_0_x_nom_2_Axolotl', 
'month_x_nom_2_Axolotl', 'nom_1_Trapezoid_x_nom_3_Russia', 'day_x_nom_2_Axolotl', 'ord_4_x_nom_1_Triangle', 
'ord_5_x_nom_3_China', 'nom_0_Red_x_nom_1_Circle', 'ord_4_x_nom_1_Polygon', 'ord_2_was_na', 'ord_5_x_day_x', 
'ord_5_x_nom_3_Finland', 'nom_4_wind_x_month_y', 'bin_1_x_nom_3_India', 'bin_2_x_nom_4_wind', 'ord_0_x_nom_1_Polygon', 
'month_x_day_y', 'month_y_x_nom_0_Red', 'ord_5', 'day_x_nom_1_Star', 'ord_4_x_nom_2_Axolotl', 'ord_0_x_nom_3_Russia', 
'bin_3_x_nom_2_Hamster', 'ord_2_x_nom_3_Russia', 'day_x_x_nom_2_Axolotl', 'month_x_nom_0_Green', 
'nom_0_Green_x_nom_2_Axolotl', 'ord_3_x_day_y']

if develop:
    train = pd.read_pickle('data/train.p')[best_features + ['target', 'id']]#.sample(frac=0.001)
else:
    train = pd.read_pickle('data/all.p')[best_features + ['target', 'id']]#.sample(frac=0.001)

validation = pd.read_pickle('data/validation.p')[best_features + ['target', 'id']]
test = pd.read_pickle('data/test.p')[best_features + ['id']]
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Columns: 190 entries, ord_2_x_ord_3 to id
dtypes: float16(134), float32(8), int16(10), int32(1), int8(37)
memory usage: 206.6 MB


In [3]:
presets = [
    {
        'model_name' : 'rf',
        'model_class' : RandomForestRegressor,
        'model_params' : {
            'n_jobs': -1,
            'n_estimators': 1000,
            
            'max_features': 0.25,
            'max_samples': 0.8,
            
            'max_depth': None,
            'min_samples_leaf': 50
        }
    },
    {
        'model_name' : 'xgb',
        'model_class' : XGBRegressor,
        'model_params' : {
            'n_jobs': -1,
            'n_estimators': 3500,
            
            'learning_rate': 0.025,
            'max_depth': 1,
            'min_child_weight': 2500,
            
            'subsample': 0.85,
            'colsample_bytree': 0.55
        },
        'fit_params' : {
            #'early_stopping_rounds': 50,
            'eval_metric': 'auc'
        }
    }
]

In [4]:
%%time

train_X = train.drop('target', axis=1)
train_y = train['target']
del train
validation_X = validation.drop('target', axis=1)
validation_y = validation['target']
del validation
predict_X = test
del test

for preset in presets:
    
    model_name = preset['model_name']
    model_class = preset['model_class']
    model_params = preset['model_params']
    
    print('Model:', model_name)
    model = model_class(**model_params)
    
    if model_class == XGBRegressor:
        fit_params = preset['fit_params']
        if 'early_stopping_rounds' in fit_params.keys():
            eval_set = [(train_X, train_y), (validation_X, validation_y)]
            model.fit(train_X, train_y, eval_set=eval_set, verbose=False, **fit_params)
        else: 
            model.fit(train_X, train_y, verbose=False, **fit_params)
    else:
        model.fit(train_X, train_y)
    
    preset['model'] = model
    
    if 'n_iter_no_change' in model_params.keys():
        print('Model contains', len(model.estimators_), 'trees')
    if model_class == XGBRegressor:
        if 'early_stopping_rounds' in fit_params.keys():
            print('Model contains', model.best_iteration+1, 'trees')
            ntree_limit = model.best_ntree_limit
        else:
            ntree_limit = model_params['n_estimators']
    
    if model_class == XGBRegressor:
        train_prediction = model.predict(train_X, ntree_limit=ntree_limit)
        validation_prediction = model.predict(validation_X, ntree_limit=ntree_limit)
    elif model_class == LogisticRegression:
        train_prediction = model.predict_proba(train_X)
        train_prediction = [x[1] for x in train_prediction]
        validation_prediction = model.predict_proba(validation_X)
        validation_prediction = [x[1] for x in validation_prediction]
    else:
        train_prediction = model.predict(train_X)
        validation_prediction = model.predict(validation_X)

    print('train ROC AUC: {}'.format(AUC(train_y, train_prediction)))
    print('validation ROC AUC: {}'.format(AUC(validation_y, validation_prediction)))
    
    if develop:
        
        if model_class == XGBRegressor:
            plot_importances(model, error=False, title=model_name)
            plot_xgboost_importances(model, model_name)

        if model_class == RandomForestRegressor:
            plot_importances(model, error=True, title=model_name)
            plot_tree_usefulness(model, validation_X, validation_y, model_name)
        
    print()

Model: rf
train ROC AUC: 0.849737992399993
validation ROC AUC: 0.8178828572130334

Model: xgb
train ROC AUC: 0.7918165984281322
validation ROC AUC: 0.7809126656786327

Wall time: 2h 21min 39s


In [5]:
for i, preset in enumerate(presets):
    
    model_class = preset['model_class']
    model_name = preset['model_name']
    model = preset['model']
    
    filename = 'data/submission_{}.csv'.format(model_name)
    print(filename)
    
    if model_class == XGBRegressor:
        if 'early_stopping_rounds' in fit_params.keys():
            ntree_limit = model.best_ntree_limit
        else:
            ntree_limit = model_params['n_estimators']
        final_prediction = model.predict(predict_X, ntree_limit=ntree_limit)
    elif model_class == LogisticRegression:
        final_prediction = model.predict_proba(predict_X)
        final_prediction = [x[1] for x in final_prediction]
    else:
        final_prediction = model.predict(predict_X)
    final_prediction = np.clip(final_prediction, 0, 1)

    prediction = predict_X[['id']]
    prediction['target'] = final_prediction
    prediction.to_csv(filename, index=False)

data/submission_rf.csv


C:\Users\Andreas\Anaconda3\envs\kaggle\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


data/submission_xgb.csv


In [6]:
"""
model = presets[1]['model']
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

features = pd.DataFrame({
    'feature': [train_X.columns[ix] for ix in indices],  
    'importance': [importances[ix] for ix in indices]})

qs = 1 - np.array([0.05, 0.1, 0.15, 0.2])
print(features['importance'].quantile(qs))

features.hist(bins=60, figsize=(18, 4))
plt.xticks(np.arange(0, 0.03, 0.001))
plt.show()

features[features['importance'] < 0.003].hist(bins=30, figsize=(18, 4))
plt.xticks(np.arange(0, 0.004, 0.0002))
plt.show()

subset = features[features['importance'] > 0.0012]
print(len(subset))
print(subset['feature'].tolist())
""";